In [4]:
from distutils.command.clean import clean

import pandas as pd
import os

# Extract function is already implemented for you
def extract(store_data, extra_data):
    extra_df = pd.read_parquet(extra_data)
    store_data = pd.read_csv(store_data)
    merged_df = store_data.merge(extra_df, on = "index")
    return merged_df

# Call the extract() function and store it as the "merged_df" variable
merged_df = extract("grocery_sales.csv", "extra_data.parquet")

In [5]:
print(merged_df.head())

   Unnamed: 0  index  Store_ID        Date  Dept  Weekly_Sales  IsHoliday  \
0           0      0         1  2010-02-05     1      24924.50          0   
1           1      1         1  2010-02-05    26      11737.12          0   
2           2      2         1  2010-02-05    17      13223.76          0   
3           3      3         1  2010-02-05    45         37.44          0   
4           4      4         1  2010-02-05    28       1085.29          0   

   Temperature  Fuel_Price  MarkDown1  MarkDown2  MarkDown3  MarkDown4  \
0        42.31       2.572        0.0        0.0        0.0        0.0   
1        42.31       2.572        0.0        0.0        0.0        0.0   
2        42.31       2.572        0.0        0.0        0.0        0.0   
3        42.31       2.572        0.0        0.0        0.0        0.0   
4        42.31       2.572        0.0        0.0        0.0        0.0   

   MarkDown5         CPI  Unemployment  Type      Size  
0        0.0  211.096358         8.

In [6]:
# Create the transform() function with one parameter: "raw_data"
def transform(raw_data):
    # Imputation steps
    raw_data["Date"] = pd.to_datetime(raw_data["Date"], format="%Y-%m-%d")

    num_cols = ["Weekly_Sales", "CPI", "Unemployment"]
    for col in num_cols:
        raw_data[col] = raw_data[col].fillna(raw_data[col].median())

    markdown_cols = ["MarkDown4", "MarkDown5"]
    raw_data[markdown_cols] = raw_data[markdown_cols].fillna(0)

    cat_cols = ["Type", "Size"]
    for col in cat_cols:
        raw_data[col] = raw_data[col].fillna(raw_data[col].mode()[0])

    # Filtering steps
    pd.to_datetime(raw_data["Date"], format="%Y-%m-%d")
    raw_data['Month'] = raw_data['Date'].dt.month

    raw_data = raw_data.loc[(raw_data["Weekly_Sales"] > 0),:]

    raw_data = raw_data[["Store_ID", "Month", "Dept", "IsHoliday", "Weekly_Sales", "CPI", "Unemployment"]]

    return raw_data

In [7]:
# Call the transform() function and pass the merged DataFrame
clean_data = transform(merged_df)
print(clean_data.head())

   Store_ID  Month  Dept  IsHoliday  Weekly_Sales         CPI  Unemployment
0         1    2.0     1          0      24924.50  211.096358         8.106
1         1    2.0    26          0      11737.12  211.096358         8.106
2         1    2.0    17          0      13223.76  211.096358         8.106
3         1    2.0    45          0         37.44  211.096358         7.484
4         1    2.0    28          0       1085.29  211.096358         7.484


In [8]:
# Create the avg_weekly_sales_per_month function that takes in the cleaned data from the last step
def avg_weekly_sales_per_month(clean_data):
    agg_data = (
        clean_data[["Month", "Weekly_Sales"]]
        .groupby("Month")["Weekly_Sales"]
        .agg("mean")
        .reset_index()
        .round(2)
        .rename(columns={"Weekly_Sales": "Avg_Sales"})
    )
    all_months = pd.DataFrame({"Month": range(1, 13)})
    agg_data = all_months.merge(agg_data, on="Month", how="left")
    agg_data["Avg_Sales"] = agg_data["Avg_Sales"].fillna(0)
    return agg_data

In [9]:
# Call the avg_weekly_sales_per_month() function and pass the cleaned DataFrame
agg_data = avg_weekly_sales_per_month(clean_data)
print(agg_data.head())

   Month  Avg_Sales
0      1   15548.42
1      2   17882.00
2      3   17168.37
3      4   17346.68
4      5   17391.83


In [10]:
# Create the load() function that takes in the cleaned DataFrame and the aggregated one with the paths where they are going to be stored
def load(full_data, full_data_file_path, agg_data, agg_data_file_path):
    full_data.to_csv(full_data_file_path, index=False)
    agg_data.to_csv(agg_data_file_path, index=False)

In [11]:
# Call the load() function and pass the cleaned and aggregated DataFrames with their paths
load(clean_data, "cleaned_data.csv", agg_data, "agg_data.csv")

In [12]:
# Create the validation() function with one parameter: file_path - to check whether the previous function was correctly executed
def validation(file_path):
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        print(f"File at {file_path} exists. Here are the first 5 rows:")
        print(df.head())
    else:
        raise Exception("File does not exist")

In [13]:
# Call the validation() function and pass first, the cleaned DataFrame path, and then the aggregated DataFrame path
validation("cleaned_data.csv")
validation("agg_data.csv")

File at cleaned_data.csv exists. Here are the first 5 rows:
   Store_ID  Month  Dept  IsHoliday  Weekly_Sales         CPI  Unemployment
0         1    2.0     1          0      24924.50  211.096358         8.106
1         1    2.0    26          0      11737.12  211.096358         8.106
2         1    2.0    17          0      13223.76  211.096358         8.106
3         1    2.0    45          0         37.44  211.096358         7.484
4         1    2.0    28          0       1085.29  211.096358         7.484
File at agg_data.csv exists. Here are the first 5 rows:
   Month  Avg_Sales
0      1   15548.42
1      2   17882.00
2      3   17168.37
3      4   17346.68
4      5   17391.83
